In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sb
%matplotlib inline

# import keras
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Add, Concatenate, BatchNormalization
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

# from keras import optimizers
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential, Model
# from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Add, Concatenate
# from keras.layers.normalization import BatchNormalization
# from keras.callbacks import ModelCheckpoint
import timeit
import os
from datetime import datetime
import tensorflow as tf
from main_v2 import generate_dataframe_from_csv3, get_model_inputs, create_multi_generator, build_model

# import wandb
# from wandb.keras import WandbCallback


In [2]:
# wandb.init(project="testing-ml")

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

df = generate_dataframe_from_csv3("train.csv")
(images, trainY) = get_model_inputs(df)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.25)

train_generator = create_multi_generator(df, train_datagen, "training") 
valid_generator = create_multi_generator(df, train_datagen, "validation") 

NameError: ("name 'gen_image_paths2' is not defined", 'occurred at index 0')

In [ ]:
display(df)

In [4]:
model = build_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputlayer_1 (InputLayer)    [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
bn_cnn_1_1 (BatchNormalizati (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
bn_cnn_2_1 (BatchNormalizati (None, 75, 75, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 128)       0     

In [5]:
from math import ceil
epochs = 20
batch_size = 32
steps = ceil(54773/batch_size)
steps_valid = ceil(18257/batch_size)

In [6]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
model.fit_generator(train_generator, 
                    steps_per_epoch=steps,
                    validation_data=valid_generator,
                    validation_steps=steps_valid,                    
                    epochs=epochs, callbacks=[checkpointer, tensorboard_callback], verbose=1)

Epoch 1/20
Found 54773 validated image filenames belonging to 1108 classes.


W0809 21:58:37.705177   392 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1711/1712 [============================>.] - ETA: 0s - loss: 16.0920 - accuracy: 9.1321e-04Found 18257 validated image filenames belonging to 1108 classes.

Epoch 00001: val_loss improved from inf to 16.09795, saving model to saved_models/weights.best.from_scratch.hdf5
1712/1712 [==============================] - 275s 161ms/step - loss: 16.0920 - accuracy: 9.1286e-04 - val_loss: 16.0980 - val_accuracy: 0.0012
Epoch 2/20
1227/1712 [====================>.........] - ETA: 1:06 - loss: 16.1019 - accuracy: 9.9328e-04

RuntimeError: Error copying tensor to device: CPU:0. GPU sync failed

In [ ]:
#!wandb run python train.py